In [45]:
import pandas as pd
from flags import FLAGS, unparsed
import numpy as np
from db.conn_db import db,cursor,engine,truncate_table,data_from_mysql,dev_id_train,get_package_dict
import operator  
from functools import reduce
import time
from sklearn.preprocessing import LabelEncoder

from scipy.sparse import csr_matrix, hstack
import xgboost as xgb
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import log_loss
from data_preprocessing import *
import datetime

In [3]:
deviceid_package_start_close=pd.read_csv(FLAGS.file_path+'deviceid_package_start_close.csv')

In [11]:
deviceid_package_start_close.head(5)

,app_id,close,device_id,start
0,1896072db9ce6406febfc17f681c2086,1489025133100,2ae,1489025123086
1,1896072db9ce6406febfc17f681c2086,1489025398515,e0450666692b72a1f580dfa082e8b2ae,1489025393513
2,1896072db9ce6406febfc17f681c2086,1489026583522,e0450666692b72a1f580dfa082e8b2ae,1489025463551
3,1896072db9ce6406febfc17f681c2086,1489034484924,e0450666692b72a1f580dfa082e8b2ae,1489034464940
4,1896072db9ce6406febfc17f681c2086,1489050201276,e0450666692b72a1f580dfa082e8b2ae,1489050176275


### app_id 打开的小时、 周几 、时长 、每个app对应的t1

In [17]:
def time_to_hour(timeStamp):
    # 字符类型的时间
    timeArray = time.localtime(int(timeStamp))
#    print(timeArray)
    # 转为时间数组
#    timeArray = time.strptime(timeArray, "%Y-%m-%d %H:%M:%S")
    return timeArray.tm_hour   # 2013

In [18]:
def def_hour_bin(hour):
    if hour>=0 and hour<2:
        return 0
    elif hour>=2 and hour<4:
        return 1
    elif hour>=4 and hour<6:
        return 2
    elif hour>=6 and hour<8:
        return 3
    elif hour>=8 and hour<10:
        return 4
    elif hour>=10 and hour<12:
        return 5
    elif hour>=12 and hour<14:
        return 6
    elif hour>=14 and hour<16:
        return 7
    elif hour>=16 and hour<18:
        return 8
    elif hour>=18 and hour<20:
        return 9
    elif hour>=20 and hour<22:
        return 10
    elif hour>=22 and hour<24:
        return 11

In [30]:
def time_to_week(timeStamp):
    timeArray =int(timeStamp)
    date = datetime.datetime.fromtimestamp(timeArray)
    day=date.weekday()
#    logging.debug(day)
    return str(day+1)   # 2013

In [20]:
def time_len(line):
    timeArray =(line['close']-line['start'])/1000
    return timeArray

In [21]:
# extract hour of the day from the timestamp 
deviceid_package_start_close['today_hour'] = deviceid_package_start_close['start'].map(lambda x:time_to_hour(x/1000))
deviceid_package_start_close['hour_bin'] = deviceid_package_start_close['today_hour'].apply(def_hour_bin)

In [25]:
deviceid_package_start_close=deviceid_package_start_close.dropna()


In [31]:
deviceid_package_start_close['week'] = deviceid_package_start_close['start'].map(lambda x:time_to_week(x/1000))


In [32]:
deviceid_package_start_close['time_len'] = deviceid_package_start_close.apply(lambda x:time_len(x),axis=1)


In [1]:
def to_t2(app):
    ret=get_package_dict(app,'t2')
    if len(ret)>0:
        return ret[0]['t2']


In [ ]:
deviceid_package_start_close['app_t2'] = deviceid_package_start_close.app_id.apply(lambda x:to_t2(x))